# Таблица default.retail

* InvoiceNo – уникальный id чека
* StockCode – Бог его знает, что это за id, использовать не будем
* Description – описание товара
* Quantity – уникальный id чека
* InvoiceDate – дата-время покупки 
* UnitPrice – цена единицы товара
* CustomerID – уникальный id покупателя
* Country – Страна

## Задача 1

Найти CustomerID, которые за февраль-март купили наибольшее количество товаров (суммарное Quantity) в англоговорящих странах, о которых у нас есть информация. Если, например, в США два покупателя купили по 100 товаров, а остальные меньше – вывести обоих.

Ожидаемый ответ:

Страна – CustomerID - Кол-во товара, которое он купил за февраль-март

## Задача 2

Вывести помесячное количество купленных будильников (в описании товара должно быть ALARM, ALARM CLOCK. Может быть еще какое-то обозначение, если найдете, будет круто :))

## Задача 3

Какой товар (какие товары, если таких с максимальной ценой несколько) в таблице является самым дорогим? Вывести id товаров и их описание.

## Задача 4

Из какой страны было больше всего покупателей?

# Таблица simulator.feed_actions

Это про действия пользователя в социальной сети, где есть Лента новостей, в которой можно совершать просмотры постов (событие view) и ставить им лайки ( событие like)

* user_id – id пользователя
* post_id – id поста
* action – действие (просмотр или лайк)
* time – дата-время события
* gender – пол
* age – возраст
* country – Страна
* city – Город
* os – Операционная система
* source – Источник трафика (откуда пользователь к нам пришел, например, "через рекламу", "сам по себе" и тд
* exp_group – экспериментальная группа (на пользователях могут проводить эксперименты, например показывать кому-то другой алгоритм рекомендаций постов)

# simulator.posts_text

* post_id – id поста
* text – текст поста
* topic – тема поста (политика, развлечения, кино)

## Задача 5

На занятии мы с вами пытались ответить на вопрос, какие посты (в зависимости от темы) набирают больше просмотров: длинные или короткие.

Измените запрос таким образом, чтобы проанализировать зависимость кол-ва просмотров от длины текста поста вне зависимости от темы. Запрос с лекции приведен ниже

In [ ]:
'''
select 
    topic,
    case when text_len <= quantile_10_len then 1
         when text_len <= quantile_20_len then 2
         when text_len <= quantile_30_len then 3
         when text_len <= quantile_40_len then 4
         when text_len <= quantile_50_len then 5
         when text_len <= quantile_60_len then 6
         when text_len <= quantile_70_len then 7
         when text_len <= quantile_80_len then 8
         when text_len <= quantile_90_len then 9
    else 10 end as length_bucket,
    avg(views) as avg_views
    
from (select texts.topic as topic,
        length(text) as text_len,
        post_data.post_id as post_id,
        post_data.views as views,
        post_data.likes as likes
    from simulator.posts_text as texts
    left join 
        (select post_id,
            countIf(action = 'view') as views,
            countIf(action = 'like') as likes
        from simulator.feed_actions
        group by post_id) as post_data
    on texts.post_id = post_data.post_id) as views_data
left join 
    (select topic,
        min(length(text)) as min_len,
        quantileExact(0.1)(length(text)) as quantile_10_len,
        quantileExact(0.2)(length(text)) as quantile_20_len,
        quantileExact(0.3)(length(text)) as quantile_30_len,
        quantileExact(0.4)(length(text)) as quantile_40_len,
        quantileExact(0.5)(length(text)) as quantile_50_len,
        quantileExact(0.6)(length(text)) as quantile_60_len,
        quantileExact(0.7)(length(text)) as quantile_70_len,
        quantileExact(0.8)(length(text)) as quantile_80_len,
        quantileExact(0.9)(length(text)) as quantile_90_len,
        max(length(text)) as max_len
    from simulator.posts_text
    group by topic) as quantile_data
using topic
group by topic, length_bucket
order by topic, length_bucket
'''

## Задача 6

Посты с какой тематикой являются самыми просматриваемыми?

## Задача 7

На сколько символов в среднем длиннее посты про политику чем посты про спорт?

## Задача 8

Постройте график конверсии из просмотров в лайки по постам про кино и про бизнес по дням. Можно ли "на глаз" сказать, что эти конверсии отличаются?

(построить суммарное кол-во лайков/суммарное кол-во просмотров group by 